# Broadcasting model, may 2016

In [1]:
#IMPORTS
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math 
import time #FOR BENCHMARKING

import sys

import filetools
import numpy as np

#MATPLOTLIB
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#PARAMETERS
class P():
    """
    General parameters for the simulation
    """
    mapSize = 50 
    
    archiveFileName = 'bcArchive' #name of the archive file used by shelve
    popArchiveName = 'bcPopArchive' #name of the population archive WITHIN shelve file
    
    gridArchiveName = 'bcLsArchive'
    sizeArchiveName = 'bcTerrainSize'
    lsArchiveFileName ='bcLsArchive'
    
    aSpeed = 1 #1 #speed of agent / round
    
    desert = 20 #below which value is a patch considered desert (used for initial placement)
    sigThreshold = 100 #used for eProg*
    
    max1sig = 1000
    max2sig = 800
    
    bumpMax = 20
    bumpWidthMin= 0.0001
    bumpWidthMax= 0.2

In [7]:
class Patches():
    """
    Class describing the landscape/patches
    """
    def __init__(self,xSize,ySize,depletion):
        self.mooreArray = np.zeros(8,dtype =[('x','i4'),('y','i4'),("height", 'f4'),("visited", 'i4')])
        self.mooreIndList = ((-1,-1),(-1,0),(-1,+1),(0,-1),(0,+1),(+1,-1),(+1,0),(+1,+1))
        
        self.xSize = xSize #size of the grid
        self.ySize = ySize
        
        #properties of a patch: x(R),y(R),height(R+),visited(R)
        self.grid = np.zeros((xSize*2+1,ySize*2+1),dtype =[('x','i4'),('y','i4'),
                                                           ('height', 'f4'),
                                                           ('visited', 'i4')])
        
        self.depletion = depletion
        
        #COORDINATES, centre of the landscape is (0,0)
        self.grid['x'] = np.indices(self.grid.shape)[0]-self.xSize
        self.grid['y'] = np.indices(self.grid.shape)[1]-self.ySize
        
        self.archive = [] #for dumping the landscape history to a file
        
        #HILLS
        self.addGaussian(25,25,1000,0.05,0.0,0.05) #0.02 #P.max1sig
        self.addGaussian(-15,-15,800,0.05,0.00,0.05) #0.02 #P.max2sig
        
        self.eMassTotal = np.sum(self.grid['height'])
    
    
    def epistemicProgressStar(self):
        """
        Returns the proportion of epistemically significant patches that have been visited.
        However, unlike in EL model, only patches above the sigLevel threshold (=100) count
        Notice that as the simulation run progresses, some patches fall below the desert threshold as agents consume significance mass 
        """
        nonDesert = self.grid[self.grid['height']>P.sigThreshold] #1-dim np-array including all patches above desert
        visitedNonDesert = nonDesert[nonDesert['visited'] == 1]
        
        numOfNonDesert = len(nonDesert)
        if numOfNonDesert != 0:
            return (len(visitedNonDesert)*1.0) / numOfNonDesert
        else:
            return 0.0
    
    
    def nonDesertPatches(self):
        nonDesert = self.grid[self.grid['height']>P.sigThreshold]
        return len(nonDesert)
        
        
    def addGaussian(self,xCen,yCen,amp,s1,s2,s3): #shape parameters s1,s2,s3
    
        indices = np.indices(self.grid.shape)
        yInd = indices[0]-self.ySize #x and y coordinate arrays
        xInd = indices[1]-self.xSize
        
        deltaY = yInd - yCen #array with y-wise distances from center of bump
        deltaX = xInd - xCen #array with x-wise distances from center of bump
       
        prod = deltaX*deltaY
        
        #self.grid['height'] += amp * np.exp(-1*(s1*deltaX**2 +s2*prod + s3*deltaY**2)) #without the rounding, there's gradient all over the terrain
        self.grid['height'] += np.round(amp * np.exp(-1*(s1*deltaX**2 +s2*prod + s3*deltaY**2)))
   
    
    def genBumps(self,bumpN,bumpMax,bumpWidthMin,bumpWidthMax):
        for bumps in range(bumpN):
            centerX,centerY = np.random.uniform(-50,50,2) #P.mapSize
            amp = np.random.uniform(0,bumpMax) #amplitude
            s1 = np.random.uniform(bumpWidthMin,bumpWidthMax) #size of the bump
            self.addGaussian(centerX,centerY,amp,s1,0,s1)
        self.eMassTotal = np.sum(self.grid['height']) #update total significance mass
    
    
    def addSineNoise(self,xCen,yCen,amp,scale):
        indices = np.indices(self.grid.shape)
        yInd = indices[0]-self.ySize #x and y coordinate arrays
        xInd = indices[1]-self.xSize
        
        deltaY = yInd - yCen #array with y-wise distance from center of bump
        deltaX = xInd - xCen
        
        self.grid['height'] += amp * np.absolute(np.cos(deltaX*1.0/scale)*np.cos(deltaY*1.0/scale)) 
        #self.grid['height'] += np.round(amp * np.exp(-1*(s1*deltaX**2 +s2*prod + s3*deltaY**2))) 
        
        self.eMassTotal = np.sum(self.grid['height']) #update total significance mass
        
        
    def intI(self,x,y): 
        """
        INPUT: integer coordinate on the landscape --> OUTPUT: integer indices to self.grid array
        """
        return x+self.xSize,y+self.ySize
    
    
    def getSig(self,x,y):
        """
        INPUT: coordinate OUTPUT: sig value
        """
        c = self.intI(x,y)
        return self.grid[c[0],c[1]]['height']
    
    
    def setSig(self,x,y,newSig):
        c = self.intI(x,y)
        self.grid[c[0],c[1]]['height'] = newSig
    
    
    def incVisit(self,x,y):
        c = self.intI(x,y)
        self.grid[c[0],c[1]]['visited'] = 1  #if value is 1, the patch has been visited
        
    
    def getPatch(self,x,y): 
        """
        returns a reference to the patch object at coordinates (x,y)
        """
        c = self.intI(x,y)
        return self.grid[c[0],c[1]] 
    
    
    def getMooreNeighborhood(self,x,y):   
        c = self.intI(x,y)
        xInd = c[0]
        yInd = c[1]
        ind = 0
        while ind < 8:
            xWrap = (xInd + self.mooreIndList[ind][0]) % (self.xSize*2+1)
            yWrap = (yInd + self.mooreIndList[ind][1]) % (self.ySize*2+1)
            self.mooreArray[ind] = self.grid[xWrap,yWrap]
            ind += 1   
        return self.mooreArray
    
    
    def deplete(self,agents):
        for a in agents:
            x = a['xP']
            y = a['yP']
            sigi = self.getSig(x,y)
            if sigi > 0:
                sigi -= sigi*self.depletion
                
            self.setSig(x,y,sigi)   
       
            
    def savePatches(self,roundN):
        self.archive.append(self.grid.copy())
        

In [4]:
class PopulationAlphas(): 
    """
    Creates a population of agents where alphas are normally distributed. 
    Ideally, this class should be combined with the one below, which implements a population with
    uniformly distributed alphas. The current solution was a shortcut that stuck.
    """
    def __init__(self,terrain,*popList): #poplist-format (size,alphaMu,alphaSd)
        self.terrain = terrain #assign a map to a population
        self.eWork = 0 #initialize epistemic work 
        self.archive = [] #a list of agent states from rounds 0 to n
        self.size = 0
        #determine total population size
        for pop in popList:
            self.size += pop[0] #pop[0]: size of subpopulation 
        
        self.agents = np.zeros((self.size),dtype=[('kind','a4'),
                                                     ('x','f4'),('y','f4'),
                                                     ('xP','i4'),('yP','i4'), #current patch as an integer
                                                     ('heading','f4'),
                                                     ('velo','f4'),
                                                     ('preSig','f4'),
                                                     ('alpha','f4')])
        
        #random heading + zero velocity
        self.agents['heading'] = np.random.uniform(0,2*math.pi,len(self.agents))
        self.agents['velo']    = 0
        self.agents['preSig']  = 0
        
        #assign kinds, can remove if not needed
        self.agents['kind'] = 'c' #controls
        
        a=0
        for pop in popList:
            self.agents[a:a+pop[0]]['alpha'] = np.random.normal(loc=pop[1],scale=pop[2],size=pop[0]) #pop[1] = mu; pop[2] = SD
            a+= pop[0] #increment index
            
        #don't let alphas be negative
        al = self.agents['alpha']
        al[al<0.01] = 0.01 

        #PLACE ALL AGENTS IN THE DESERT
        for a in self.agents:
            low = False
            while not low:
                a['x'] = np.random.uniform(-self.terrain.xSize,self.terrain.xSize,len(self.agents)).astype(np.float32)
                a['y'] = np.random.uniform(-self.terrain.ySize,self.terrain.ySize,len(self.agents)).astype(np.float32)
                if self.terrain.getSig(a['x'],a['y']) < P.desert: 
                    low = True

    
    def incEpistemicWorkAndDeplete(self):
        eWorkCount = 0
        for a in self.agents:
            sig = self.terrain.getSig(a['xP'],a['yP'])
            if sig >0:
                sInc = sig*self.terrain.depletion
                self.eWork += sInc #increase eWork by population
                self.terrain.setSig(a['xP'],a['yP'],sig-sInc) #deplete patch

    
    def findPatches(self):
        #beginning of each round, calculate patches for all agents, reduces unnecessary np.round()s
        self.agents['xP'] = np.round(self.agents['x'])
        self.agents['yP'] = np.round(self.agents['y'])
         
        
    def wrap(self,coord,limit):
        """
        Couldn't figure out how to do modulo wrapping for non-integer values, so wrote this
        """
        if coord > limit:
            newCoord = coord-2*limit
        elif coord < -limit:
            newCoord = coord+2*limit
        else: newCoord = coord
        return newCoord
    

    def move(self,i):
        agent = self.agents[i]
        agent['x'] += np.cos(agent['heading'])*agent['velo']
        agent['y'] += np.sin(agent['heading'])*agent['velo']
        
        agent['x'] = self.wrap(agent['x'],self.terrain.xSize)
        agent['y'] = self.wrap(agent['y'],self.terrain.ySize)
    
    
    def setAgentPosition(self,i,x,y):
        agent = self.agents[i]
        agent['x'] = x
        agent['y'] = y

    
    def setHeadingToPatch(self,i,xTarg,yTarg):
        """
        sets agent's heading towards the center of a target patch
        takes fastest route, i.e. takes wrapping into account
        """
        agent = self.agents[i]
        
        #for both x and y, there are three cases to consider: actual value x/yTarg and its projections on both sides
        xTarget = np.array([0,0,0])
        xTarget[0] = xTarg-self.terrain.xSize*2
        xTarget[1] = xTarg #the actual point
        xTarget[2] = xTarg+self.terrain.xSize*2 #projection to the right
        cosArray = xTarget-agent['x']
        minInd = np.argmin(np.absolute(cosArray)) #get index for the point at shortest distance
        cos = cosArray[minInd]
        
        yTarget = np.array([0,0,0])
        yTarget[0] = yTarg-self.terrain.ySize*2
        yTarget[1] = yTarg #the actual point
        yTarget[2] = yTarg+self.terrain.ySize*2 
        sinArray = yTarget-agent['y']
        minInd = np.argmin(np.absolute(sinArray)) #get index for the point at shortest distance
        sin = sinArray[minInd]
        
        #check for division by zero
        if cos == 0:
            if sin > 0: agent['heading'] = math.pi/2
            else: agent['heading'] = 3*math.pi/2
        else:
            tan = sin / cos
            #choose the heading in the correct quadrant
            if cos > 0:
                agent['heading'] = np.arctan(tan) % (2*np.pi) #modulo makes the angle > 0
            else:
                agent['heading'] = (np.pi+np.arctan(tan))%(2*np.pi)
  
            
    def moveAgent(self,i):    
        agent = self.agents[i]    
        self.terrain.incVisit(agent['xP'],agent['yP'])
        sig = self.terrain.getSig(agent['xP'],agent['yP'])
        agentX = agent['x']
        agentY = agent['y']
    
        #GOING DOWNHILL?
        if sig < agent['preSig']:
            #1. SOCIAL LEARNING:
            distX1 = self.agents['x']-agentX
            distX2 = (2*self.terrain.xSize) - distX1 #WRAPPED DISTANCE
            distY1 = self.agents['y']-agentY
            distY2 = (2*self.terrain.ySize) - distY1
            distX = np.minimum(distX1,distX2)
            distY = np.minimum(distY1,distY2)
            
            heightDeltas = self.agents['preSig']-sig #COMPARE YOUR OWN ELEVATION TO HEIGHT OF OTHERS AT LAST TIMESTEP
            
            distX[i] = np.nan #10000 #don't follow yourself
            distY[i] = np.nan #10000
            
            dist = np.sqrt(distX**2 + distY**2)
            
            denominator = dist
            
            inclines = heightDeltas / denominator 
            
            #agentsInCone = self.agents[inclines > agent['alpha']] #CHOOSE THOSE AGENTS WHO ARE ABOVE THE REQUIRED ANGLE (TAN A) = ALPHA
            
            maxIncline = np.nanmax(inclines) #max social incline
            #print "maxIncline", maxIncline
           
            if maxIncline > agent['alpha']:
                #print "following the leader"
                maxAgent = self.agents[np.nanargmax(inclines)]
                self.setHeadingToPatch(i,maxAgent['x'],maxAgent['y'])
                agent['velo']=1.0
            else:
                #IF NO OTHER AGENTS IN CONE --> INDIVIDUAL SEARCH
                nh = self.terrain.getMooreNeighborhood(agent['xP'],agent['yP'])
                
                #compare heights
                mooreElevations = nh['height']-sig 
                geqMoores = nh[mooreElevations >= 0] #tsekkaa taa, onhan boolean indexing oikein? pitais palautta lista ylempana tai samalla tasolllla olevista napureista
                if len(geqMoores) > 0:
                    chosenPatch = np.random.choice(geqMoores) #choose a new (geq) patch randomdly)
                    agent['velo'] = 1.0
                    self.setHeadingToPatch(i,chosenPatch['x'],chosenPatch['y'])
                else:
                    #IF ONLY LOWER PATCHES, STOP:
                    agent['velo'] = 0.0
                    #print "stopping"
        
        else: #MOVING UPHILL, NO NEED TO UPDATE HEADING
            agent['velo'] = 1.0
        
        #MOVE
        agent['preSig'] = sig
        self.move(i)      
                   
    
    def savePop(self,roundN):
        self.archive.append(self.agents.copy())
    
    
    def popAveHeights(self):
        return np.mean(self.agents['preSig']) #using the preSig is not conceptually precise. But the bias is small.  

In [5]:
class PopulationUniform(): 
    """
    A population with uniformly distributed alphas
    """
    def __init__(self,size,alphaMin,alphaMax,terrain):
        self.size = size 
        self.alphaMin= alphaMin
        self.alphaMax = alphaMax
        
        self.terrain = terrain #assign a map to a population
        self.eWork = 0 #initialize epistemic work
        self.archive = [] #a list of agent states from rounds 0 to n
        self.agents = np.zeros((self.size),dtype=[('kind','a4'),
                                                     ('x','f4'),('y','f4'),
                                                     ('xP','i4'),('yP','i4'), #current patch as an integer
                                                     ('heading','f4'),
                                                     ('velo','f4'),
                                                     ('preSig','f4'),
                                                     ('alpha','f4')])
        #assign kinds, can be removed if not needed
        self.agents['kind'] = 'c' #controls
                
        #random heading + zero velocity
        self.agents['heading'] = np.random.uniform(0,2*math.pi,len(self.agents))
        self.agents['velo']    = 0
        self.agents['preSig']  = 0
        
        #UNIFORM DISTRIBUTION FOR ALPHAS
        self.agents['alpha'] = np.random.uniform(low=self.alphaMin,high=self.alphaMax,size=self.size) 
        
        #PLACE ALL AGENTS IN THE DESERT
        for a in self.agents:
            low = False
            while not low:
                a['x'] = np.random.uniform(-self.terrain.xSize,self.terrain.xSize,len(self.agents)).astype(np.float32)
                a['y'] = np.random.uniform(-self.terrain.ySize,self.terrain.ySize,len(self.agents)).astype(np.float32)
                if self.terrain.getSig(a['x'],a['y']) < P.desert: 
                    low = True
    
    
    def incEpistemicWorkAndDeplete(self):
        eWorkCount = 0
        for a in self.agents:
            sig = self.terrain.getSig(a['xP'],a['yP'])
            if sig >0:
                sInc = sig*self.terrain.depletion
                self.eWork += sInc #increase eWork by population
                self.terrain.setSig(a['xP'],a['yP'],sig-sInc) #deplete patch
    
    
    def findPatches(self):
        #beginning of each round, calculate patches for all agents, reduces unnecessary np.round()s
        self.agents['xP'] = np.round(self.agents['x'])
        self.agents['yP'] = np.round(self.agents['y'])
        
        
    def wrap(self,coord,limit):
        """
        Couldn't figure out how to do module wrapping for real values, so wrote this
        """
        if coord > limit:
            newCoord = coord-2*limit
        elif coord < -limit:
            newCoord = coord+2*limit
        else: newCoord = coord
        return newCoord
    

    def move(self,i):
        agent = self.agents[i]
        agent['x'] += np.cos(agent['heading'])*agent['velo']
        agent['y'] += np.sin(agent['heading'])*agent['velo']
        
        agent['x'] = self.wrap(agent['x'],self.terrain.xSize)
        agent['y'] = self.wrap(agent['y'],self.terrain.ySize)
    
    
    def setAgentPosition(self,i,x,y):
        agent = self.agents[i]
        agent['x'] = x
        agent['y'] = y

    
    def setHeadingToPatch(self,i,xTarg,yTarg):
        """
        sets agent's heading towards the center of a target patch
        takes fastest route, i.e. takes wrapping into account
        """
        agent = self.agents[i]
        
        #for both x and y, there are three cases to consider: actual value x/yTarg and its projections on both sides
        xTarget = np.array([0,0,0])
        xTarget[0] = xTarg-self.terrain.xSize*2
        xTarget[1] = xTarg #the actual point
        xTarget[2] = xTarg+self.terrain.xSize*2 #projection to the right
        cosArray = xTarget-agent['x']
        minInd = np.argmin(np.absolute(cosArray)) #get index for the point at shortest distance
        cos = cosArray[minInd]
        
        yTarget = np.array([0,0,0])
        yTarget[0] = yTarg-self.terrain.ySize*2
        yTarget[1] = yTarg #the actual point
        yTarget[2] = yTarg+self.terrain.ySize*2 
        sinArray = yTarget-agent['y']
        minInd = np.argmin(np.absolute(sinArray)) #get index for the point at shortest distance
        sin = sinArray[minInd]
        
        #check for division by zero
        if cos == 0:
            if sin > 0: agent['heading'] = math.pi/2
            else: agent['heading'] = 3*math.pi/2
        else:
            tan = sin / cos
            #choose the heading in the correct quadrant
            if cos > 0:
                agent['heading'] = np.arctan(tan) % (2*np.pi) #modulo makes the angle > 0
            else:
                agent['heading'] = (np.pi+np.arctan(tan))%(2*np.pi)
  
            
    def moveAgent(self,i):    
        agent = self.agents[i]    
        self.terrain.incVisit(agent['xP'],agent['yP'])
        sig = self.terrain.getSig(agent['xP'],agent['yP'])
        agentX = agent['x']
        agentY = agent['y']
    
        #GOING DOWNHILL?
        if sig < agent['preSig']:
            #1. SOCIAL LEARNING:
            distX1 = self.agents['x']-agentX
            distX2 = (2*self.terrain.xSize) - distX1 #WRAPPED DISTANCE
            distY1 = self.agents['y']-agentY
            distY2 = (2*self.terrain.ySize) - distY1
            distX = np.minimum(distX1,distX2)
            distY = np.minimum(distY1,distY2)
            
            heightDeltas = self.agents['preSig']-sig #COMPARE YOUR OWN ELEVATION TO HEIGHT OF OTHERS AT LAST TIMESTEP
            
            distX[i] = np.nan #10000 #don't follow yourself
            distY[i] = np.nan #10000
            
            dist = np.sqrt(distX**2 + distY**2)
           
            denominator = dist
            
            inclines = heightDeltas / denominator #THE LINEAR FORMULA FOR ALPHAS
            
            #agentsInCone = self.agents[inclines > agent['alpha']] #CHOOSE THOSE AGENTS WHO ARE ABOVE THE REQUIRED ANGLE (TAN A) = ALPHA
            
            maxIncline = np.nanmax(inclines) #max social incline
            #print "maxIncline", maxIncline
           
            if maxIncline > agent['alpha']:
                #print "following the leader"
                maxAgent = self.agents[np.nanargmax(inclines)]
                self.setHeadingToPatch(i,maxAgent['x'],maxAgent['y'])
                agent['velo']=1.0
            else:
                #IF NO OTHER AGENTS IN CONE --> INDIVIDUAL SEARCH
                nh = self.terrain.getMooreNeighborhood(agent['xP'],agent['yP'])
                
                #compare heights
                mooreElevations = nh['height']-sig 
                geqMoores = nh[mooreElevations >= 0] #tsekkaa taa, onhan boolean indexing oikein? pitais palautta lista ylempana tai samalla tasolllla olevista napureista
                if len(geqMoores) > 0:
                    chosenPatch = np.random.choice(geqMoores) #choose a new (geq) patch randomdly)
                    agent['velo'] = 1.0
                    self.setHeadingToPatch(i,chosenPatch['x'],chosenPatch['y'])
                else:
                    #IF ONLY LOWER PATCHES, STOP:
                    agent['velo'] = 0.0
                    #print "stopping"
        
        else: #MOVING UPHILL, NO NEED TO UPDATE HEADING
            agent['velo'] = 1.0
        
        #MOVE
        agent['preSig'] = sig
        self.move(i)      
                   
    
    def savePop(self,roundN):
        self.archive.append(self.agents.copy())
    
    
    def popAveHeights(self):
        return np.mean(self.agents['preSig']) #using the preSig is not conceptually precise. But a useful approximation    

In [6]:
def runRound(i,save=False):
    i = 0
    
    #move agents
    while i < len(pop.agents):
        pop.moveAgent(i)
        i += 1
    pop.findPatches()
    pop.incEpistemicWorkAndDeplete()
    if save: 
        pop.savePop(i) #archive population state
    
    #ls.savePatches(i) #save landscape state

In [ ]:
### EXP LAB. TRY EVERYTHING OUT HERE FIRST ###

#SETUP SIMULATION
bumps = 200 
depe = 0
rounds = 1000 

#CREATE LANDSCAPE
ls = Patches(P.mapSize,P.mapSize,depe)
ls.genBumps(bumps,P.bumpMax,P.bumpWidthMin,P.bumpWidthMax)

print "Ave patch height:", (np.sum(ls.grid['height'])/(101*101))

#CREATE POPULATION
pop = PopulationAlphas(ls,(45,100,1),(5,100,1))
#pop = PopulationAlphas(ls,(40,1,1),(10,100,1)) #poplist-format: (size,alphaMu,alphaSd)
#pop = PopulationUniform(50,0.1,100,ls)
pop.findPatches()

eWorkArch  = []
aveHeights = []
eProgArch  = [] 

#dump the landscape to a file
filetools.shelveDump(ls.grid,P.gridArchiveName,P.lsArchiveFileName)
filetools.shelveDump(ls.xSize,P.sizeArchiveName,P.lsArchiveFileName)

    
#RUN SIMULATION
print ""
print "SIMULATION START"
startTime = time.clock()
#sys.stdout.write("round: \n")
for roundNum in range(rounds):
    roundText = "\n"+str(roundNum) + "\n"
    #sys.stdout.write(roundText)
    runRound(roundNum,save=True) 
    eWorkArch.append(pop.eWork/ls.eMassTotal) #ls.eMassTotal
    aveHeights.append(pop.popAveHeights())
    eProgArch.append(ls.epistemicProgressStar())
    #print "eWork: ",pop.eWork
    #print "eProg*: ",ls.epistemicProgressStar()
    
finishTime = time.clock()
print "Time for one round:",(finishTime-startTime)/rounds
print "Total time:", (finishTime-startTime)

print "eMassTotal",ls.eMassTotal
print "eWork",pop.eWork
print "Proportion of epistemic mass collected: ", (pop.eWork/ls.eMassTotal) 

#dump the population history to file
filetools.shelveDump(pop.archive,P.popArchiveName,P.archiveFileName)

#PLOTTING
roundN = list(range(rounds))

plt.figure()
print len(eWorkArch)
#plt.plot(numberOfBumps,eProg,'r',label='Epistemic progress')
plt.plot(roundN,eWorkArch,'b',label='EW_pop/t')
plt.legend(loc='upper left')
plt.xlabel('round')
plt.ylabel('eWork')
plt.ylim([0.0,1.0])
#plt.suptitle('Average epistemic significance of the population')
#plt.savefig('diverse_learners_on_bumpy.pdf')

plt.show()

plt.figure()

plt.plot(roundN,eProgArch,'b',label='eProg*/t')
plt.legend(loc='upper left')
plt.xlabel('round')
plt.ylabel('eWork')
plt.ylim([0.0,1.0])
#plt.suptitle('Average epistemic significance of the population')
#plt.savefig('diverse_learners_on_bumpy.pdf')

plt.show()

In [ ]:
### (1) PERFORMANCE ON A SMOOTH/RUGGED LANDSCAPE ###

#PARAMETERS
muValues = [0.1,1,10,20,50,100]
SDValues = [0.1,1,10]
depValues = [0,0.001,0.01,0.1,1]


#DUMMIES FOR RUNNING WITH UNIFORM DISTR. UNCOMMENT FOR THAT
#muValues = [1]
#SDValues = [1]
#FOR UNIFORM
alphaMin = 1
alphaMax = 100


sampleSize = 50
popSize = 50
roundsMax = 1000

bumps = 300 #0,50,100,200 or 300


eWork200Table = np.zeros((len(muValues),len(SDValues))) #store here results from simulation
eWork400Table = np.zeros((len(muValues),len(SDValues)))
eWork1000Table = np.zeros((len(muValues),len(SDValues))) 

eProg200Table = np.zeros((len(muValues),len(SDValues)))
eProg400Table = np.zeros((len(muValues),len(SDValues)))
eProg1000Table = np.zeros((len(muValues),len(SDValues)))

print ""
print "SIMULATION START"

startTime = time.clock()

for dep in depValues:
    print ('Depletion: %f' %dep)
    
    indMu = 0
    for a in muValues:
        indSD = 0
        for sd in SDValues:
            print ('mu: %f, SD: %f' %(a,sd))

            sample200 = []
            sample400 = []
            sample1000 = []
            
            eProgSample200 = []
            eProgSample400 = []
            eProgSample1000 = []

            for sample in range(sampleSize):
                ls = Patches(P.mapSize,P.mapSize,dep)
                ls.genBumps(bumps,P.bumpMax,P.bumpWidthMin,P.bumpWidthMax) 
                
                #choose there the right distribution!
                pop = PopulationAlphas(ls,(popSize,a,sd))
                #pop = PopulationUniform(popSize,alphaMin,alphaMax,ls)
                
                pop.findPatches()

                rounds = roundsMax #how many rounds

                for roundNum in range(rounds):
                    runRound(roundNum) 

                    #RECORD PROGRESS
                    if roundNum == 199:
                        sample200.append(pop.eWork/ls.eMassTotal)
                        eProgSample200.append(ls.epistemicProgressStar())
                    if roundNum == 399:
                        sample400.append(pop.eWork/ls.eMassTotal)
                        eProgSample400.append(ls.epistemicProgressStar())
                    if roundNum == 999:
                        sample1000.append(pop.eWork/ls.eMassTotal)
                        eProgSample1000.append(ls.epistemicProgressStar())
            
            #CALCULATE SAMPLE MEANS
            eWork200Table[indMu,indSD] = np.mean(sample200)
            eWork400Table[indMu,indSD] = np.mean(sample400)
            eWork1000Table[indMu,indSD] = np.mean(sample1000)
            
            eProg200Table[indMu,indSD] = np.mean(eProgSample200)
            eProg400Table[indMu,indSD] = np.mean(eProgSample400)
            eProg1000Table[indMu,indSD] = np.mean(eProgSample1000)
            
            indSD +=1

        indMu += 1

    #print " "
    print "eWork at 200" 
    print eWork200Table
    print "eWork at 400" 
    print eWork400Table
    print "eWork at 1000" 
    print eWork1000Table
    print " "
    
    print "eProg at 200" 
    print eProg200Table
    print "eProg at 400" 
    print eProg400Table
    print "eProg at 1000" 
    print eProg1000Table
    print " "
    
    #DUMP TO CSV
    np.savetxt(str("rugged_uniform_B300_"+str(dep)+"_eWorkAt200.csv"), eWork200Table, delimiter=",", fmt='%.8f') #formatting: print 8 digits after the decimal point
    np.savetxt(str("rugged_uniform_B300_"+str(dep)+"_eWorkAt400.csv"), eWork400Table, delimiter=",", fmt='%.8f')
    np.savetxt(str("rugged_uniform_B300_"+str(dep)+"_eWorkAt1000.csv"), eWork1000Table, delimiter=",", fmt='%.8f')
    
    np.savetxt(str("rugged_uniform_B300_"+str(dep)+"_eProgAt200.csv"), eProg200Table, delimiter=",", fmt='%.8f')
    np.savetxt(str("rugged_uniform_B300_"+str(dep)+"_eProgAt400.csv"), eProg400Table, delimiter=",", fmt='%.8f')
    np.savetxt(str("rugged_uniform_B300_"+str(dep)+"_eProgAt1000.csv"), eProg1000Table, delimiter=",", fmt='%.8f')
    

In [ ]:
### (2) ADD RUGGEDNESS ###

#SETUP SIMULATION
bumpVals = np.arange(start=0,stop=301,step=10)

bumpMax = 20 #
depe = 0.01
sampleSize = 50 

popSize = 50 
roundsMax = 1000

#AGENT PROPERTIES
#alphaMu = 1 #1 --> SOCIAL; 100 --> INDIVIDUAL
#alphaSD = 1

uniMin = 1
uniMax = 100

eWork200Table = np.zeros((len(bumpVals))) #store here results from simulation
eWork400Table = np.zeros((len(bumpVals))) #store here results from simulation
eWork1000Table = np.zeros((len(bumpVals))) #store here results from simulation

eWstd200 = np.zeros((len(bumpVals)))
eWstd400 = np.zeros((len(bumpVals)))
eWstd1000 = np.zeros((len(bumpVals)))

#eProg*
eProg200Table = np.zeros((len(bumpVals)))
eProg400Table = np.zeros((len(bumpVals)))
eProg1000Table = np.zeros((len(bumpVals)))

print ""
print "SIMULATION START"

startTime = time.clock()

indBumps = 0
for bumps in bumpVals:
    print ('Bumps: %i' %bumps)
    
    eWsample200 = []
    eWsample400 = []
    eWsample1000 = []
    
    eProgSample200 = []
    eProgSample400 = []
    eProgSample1000 = []
    
    for sample in range(sampleSize):
        ls = Patches(P.mapSize,P.mapSize,depe)
        ls.genBumps(bumps,P.bumpMax,P.bumpWidthMin,P.bumpWidthMax)

        #UNCOMMMENT THE CORRECT LINE
        pop = PopulationAlphas(ls,(popSize,alphaMu,alphaSD)) #choose there the right distribution!
        #pop = PopulationUniform(popSize,uniMin,uniMax,ls)
        
        pop.findPatches()

        rounds = roundsMax #how many rounds

        for roundNum in range(rounds):
            runRound(roundNum) 

            if roundNum == 199:
                eWsample200.append(pop.eWork/ls.eMassTotal)
                eProgSample200.append(ls.epistemicProgressStar())
            if roundNum == 399:
                eWsample400.append(pop.eWork/ls.eMassTotal)
                eProgSample400.append(ls.epistemicProgressStar())
            if roundNum == 999:
                eWsample1000.append(pop.eWork/ls.eMassTotal)
                eProgSample1000.append(ls.epistemicProgressStar())

    eWork200Table[indBumps] = np.mean(eWsample200)
    eWstd200[indBumps] = np.std(eWsample200)
    eWork400Table[indBumps] = np.mean(eWsample400)
    eWstd400[indBumps] = np.std(eWsample400)
    eWork1000Table[indBumps] = np.mean(eWsample1000)
    eWstd1000[indBumps] = np.std(eWsample1000)
    
    eProg200Table[indBumps]  = np.mean(eProgSample200)
    eProg400Table[indBumps]  = np.mean(eProgSample400)
    eProg1000Table[indBumps] = np.mean(eProgSample1000)
    
    indBumps += 1


#PRINT RESULTS
print ls.eMassTotal    
print pop.eWork

print " "
print "eWork at 200" 
print list(eWork200Table)
#print list(eWstd200)
print "eWork at 400" 
print list(eWork400Table)
#print list(eWstd400)
print "eWork at 1000" 
print list(eWork1000Table)
#print list(eWstd1000)
print " "

#DUMP TO CSV. CHOOSE THE RIGHT FILE NAME! IND/SOC/UNI
np.savetxt("uni_eWorkAt200_addingBumps.csv", eWork200Table, delimiter=",", fmt='%.8f')
np.savetxt("uni_eWorkAt400_addingBumps.csv", eWork400Table, delimiter=",", fmt='%.8f')
np.savetxt("uni_eWorkAt1000_addingBumps.csv", eWork1000Table, delimiter=",", fmt='%.8f')

np.savetxt("uni_eProgAt200_addingBumps.csv", eProg200Table, delimiter=",", fmt='%.8f')
np.savetxt("uni_eProgAt400_addingBumps.csv", eProg400Table, delimiter=",", fmt='%.8f')
np.savetxt("uni_eProgAt1000_addingBumps.csv", eProg1000Table, delimiter=",", fmt='%.8f')


In [ ]:
### MAY 2016. ADDING INDIVIDUALISTS TO SOCIAL POP, LAMBDA 0.01 ###
### Used also to create the plot for non-desert patches (supplementary materials)

#create landscape & population
depe = 0.01
sampleSize = 50
rounds = 1000 #how many rounds
bumps = 100  
 

eWorkTable = np.zeros((sampleSize,rounds))
eWorkFinal = np.zeros(rounds)
eWorkStd = np.zeros(rounds)

eProgTable = np.zeros((sampleSize,rounds))
eProgFinal = np.zeros(rounds)

ndPatchesTable = np.zeros((sampleSize,rounds))
nonDesertPatches = np.zeros(rounds) #keep track of how many non-desert patches remain

numberOfInds = [0,10,20,30,40,50]
numberOfInds = [0,10,50] #for tracking non-desert patches

for inds in numberOfInds:
    print ("Inds: %i" %inds)
    for sample in range(sampleSize):
        ls = Patches(P.mapSize,P.mapSize,depe)
        ls.genBumps(bumps,P.bumpMax,P.bumpWidthMin,P.bumpWidthMax)

        pop = PopulationAlphas(ls,(50-inds,0.1,1),(inds,100,1))
        pop.findPatches()
        
        #RUN SIMULATION
        #print ""
        #print "SIMULATION START"
        startTime = time.clock()
        #sys.stdout.write("round: \n")
        for roundNum in range(rounds):
            roundText = str(roundNum) + "\t"
            #sys.stdout.write(roundText)
            runRound(roundNum) 
            eWorkTable[sample,roundNum]=(pop.eWork)/ls.eMassTotal
            eProgTable[sample,roundNum]=ls.epistemicProgressStar()
            ndPatchesTable[sample,roundNum] = ls.nonDesertPatches()
            
        #print "Proportion of epistemic mass collected: ", (pop.eWork)/ls.eMassTotal
        #print "eMassTotal",ls.eMassTotal
        #print "eWork",pop.eWork

    eWorkFinal = list(np.mean(eWorkTable,axis=0))
    eWorkStd = list(np.std(eWorkTable,axis=0))

    eProgFinal = list(np.mean(eProgTable,axis=0))
    
    nonDesertPatches = list(np.mean(ndPatchesTable,axis=0))
    
    #print eWorkFinal
    #print eWorkStd
    
    #DUMP TO CSV
    #np.savetxt(str("adding_"+str(inds)+"_inds__B300_eWork.csv"), eWorkFinal, delimiter=",", fmt='%.8f') 
    #np.savetxt(str("adding_"+str(inds)+"_inds__B300_eProg.csv"), eProgFinal, delimiter=",", fmt='%.8f') 
    
    np.savetxt(str("adding_"+str(inds)+"_b100_nonDesertPatches.csv"), nonDesertPatches, delimiter=",", fmt='%.8f') 

print "Done"    